Source: https://github.com/oschow/take-a-hike/blob/master/AllTrails/scrape_clean/scrape_ratings.py

In [ ]:
# installs
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 5.6MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/

In [ ]:
# imports
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, SoupStrainer
from pymongo import MongoClient
import time
import sys
import re
import urllib

In [ ]:
# Set up chromedriver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [ ]:
# Definition for logging in
def login(browser):
    browser.get('https://www.alltrails.com/login?ref=header')
    username = browser.find_element_by_name("userEmail")
    password = browser.find_element_by_name("userPassword")
    username.send_keys("221ornithomimus@gmail.com")
    password.send_keys("Dj3!120423u_9jkd_5!k34")
    browser.find_element_by_xpath("//input[@data-test-id='formButton-submit']").click()


# Scroll through the page and load all of the hikes
def get_hikes(browser):
    browser.get('https://www.alltrails.com/canada/british-columbia')
    # while True:
    #     try:
    #         load_more_hikes = WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.XPATH,"//div[@id='load_more'] [@class='feed-item load-more trail-load'][//a]")))
    #         load_more_hikes.click()
    #         time.sleep(7)
    #     except:
    #         break
    soup = BeautifulSoup(browser.page_source)
    return soup

def get_photos(hike, browser):
    browser.get(hike[0])
    soup2 = BeautifulSoup(browser.page_source)
    hikes = soup2.find_all(class_="photo-item gallery sm")[:3] # get first 3 photos
    print(hikes)
    i = 0
    for element in hikes:
        hike_url = element.get('href') # get url from hike
        urllib.request.urlretrieve(hike_url, filename="/content/hike_photos/" + hike[1] + "-" + str(i) + ".jpg")
        i += 1
        time.sleep(5) # wait 5 seconds to avoid being blocked

In [ ]:
# Log in to AllTrails
login(wd)

In [ ]:
# Scroll through and load all available hikes
soup = get_hikes(wd)

hikes = soup.find_all(itemid=re.compile('/trail/\S*'))

In [ ]:
# Get a list of hike urls and their id numbers
hike_links = []

for hike in hikes:
    hike_links += [('https://www.alltrails.com' + hike.get('itemid') + '/photos', hike.get('data-id'))]

In [ ]:
# Download each image in the list
for hike in hike_links:
    get_photos(hike,wd)

In [ ]:
wd.get(hike_links[0][0])
soup2 = BeautifulSoup(wd.page_source)
# hikes = soup2.find_all(class='photo-item gallery sm')

MaxRetryError: ignored

In [ ]:
photos = soup2.find_all(class_="photo-item gallery sm")[:5]

In [ ]:
photos

[<a class="photo-item gallery sm" data-credit-link="" data-photo-id="23957167" href="https://cdn-assets.alltrails.com/uploads/photo/image/23957167/extra_large_04c8a06a56c3a328db71d851edb1bfb7.jpg"><img class="trail-photo lazyloaded" data-src="https://cdn-assets.alltrails.com/uploads/photo/image/23957167/large_04c8a06a56c3a328db71d851edb1bfb7.jpg" id="trail-photo-23957167" src="https://cdn-assets.alltrails.com/uploads/photo/image/23957167/large_04c8a06a56c3a328db71d851edb1bfb7.jpg"/></a>,
 <a class="photo-item gallery sm" data-credit-link="" data-photo-id="34480541" href="https://cdn-assets.alltrails.com/uploads/photo/image/34480541/extra_large_fcf2776da5f128d9e494e707ba167708.jpg"><img class="trail-photo lazyloaded" data-src="https://cdn-assets.alltrails.com/uploads/photo/image/34480541/large_fcf2776da5f128d9e494e707ba167708.jpg" id="trail-photo-34480541" src="https://cdn-assets.alltrails.com/uploads/photo/image/34480541/large_fcf2776da5f128d9e494e707ba167708.jpg"/></a>,
 <a class="pho

In [ ]:
# Go to page
wd.get("https://www.alltrails.com/canada/british-columbia")

# For each entry, get itemid
soup = BeautifulSoup(wd.page_source)

hikes = soup.select('div data-hydrate')

In [ ]:
hikes = soup.select('div data-hydrate')

In [ ]:
hikes

[]